In [1]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

In [2]:
# Vaccine Location (Path) and read the CSV File
VaccineLocation_path = "/Users/kellenquinn/Desktop/untitled folder/vaccinelocations.csv"
VaccineLocation_df = pd.read_csv(VaccineLocation_path)

In [3]:
#COVID Cases by Zipcode 
csvfile1 = "/Users/kellenquinn/Desktop/project1-collab/Resources/AZ_Covid Cases.csv"
cases_df = pd.read_csv(csvfile1)
cases_df

,POSTCODE,ConfirmedCaseCategory,ConfirmedCaseCount
0,85003,Greater Than 1000,1534
1,85004,501-1000,969
2,85006,Greater Than 1000,3272
3,85007,Greater Than 1000,2424
4,85008,Greater Than 1000,9079
...,...,...,...
405,86514 Tribal,Data Suppressed,Data Suppressed
406,86515,0,0
407,86515 Tribal,Data Suppressed,Data Suppressed
408,86538 Tribal,Data Suppressed,Data Suppressed


In [22]:
finalcases_df = cases_df.dropna()
Clean_df = finalcases_df.drop(columns='ConfirmedCaseCategory')
Clean_df.loc[~(Clean_df['ConfirmedCaseCount']=='Data Suppressed')]
Final_df = Clean_df.loc[~(Clean_df['ConfirmedCaseCount']=='Data Suppressed')]
Final_df

,POSTCODE,ConfirmedCaseCount
0,85003,1534
1,85004,969
2,85006,3272
3,85007,2424
4,85008,9079
...,...,...
397,86505,137
399,86507,0
402,86511,1-10
404,86514,0


In [35]:
#Continue Clean-up of Data
Final_df.loc[Final_df['ConfirmedCaseCount']=='1-10','ConfirmedCaseCount'] = 5
Final_df


/Users/kellenquinn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-35-ee951bc4c75f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_df.loc[Final_df['ConfirmedCaseCount']=='1-10','ConfirmedCaseCount'] = 5


,POSTCODE,ConfirmedCaseCount,Lat,Lon
0,85003,1534,33.4496,-112.079
1,85004,969,33.4528,-112.069
2,85006,3272,33.4636,-112.054
3,85007,2424,33.4463,-112.089
4,85008,9079,33.466,-112.012
...,...,...,...,...
397,86505,137,35.5677,-109.664
399,86507,0,36.4107,-109.259
402,86511,5,35.7497,-109.331
404,86514,0,36.9182,-109.331


In [23]:
Final_df["Lat"] = ""
Final_df["Lon"] = ""
Final_df

<ipython-input-23-386bb5425e56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_df["Lat"] = ""
<ipython-input-23-386bb5425e56>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_df["Lon"] = ""


,POSTCODE,ConfirmedCaseCount,Lat,Lon
0,85003,1534,,
1,85004,969,,
2,85006,3272,,
3,85007,2424,,
4,85008,9079,,
...,...,...,...,...
397,86505,137,,
399,86507,0,,
402,86511,1-10,,
404,86514,0,,


In [24]:
# Query URL
query_url = "https://maps.googleapis.com/maps/api/geocode/json?"

# Lat Lon for Zipcodes
for index, zips in Final_df.iterrows():
    params= {
        "key": g_key,
        "address": zips["POSTCODE"],
     }

    response = requests.get(query_url, params = params).json()

    try: 
        Final_df.loc[index, "Lat"]=response["results"][0]["geometry"]["location"]["lat"]
        Final_df.loc[index, "Lon"]=response["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print(f"Error Occured, Skipping Zip Code")

Final_df.head()        


/Users/kellenquinn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-24-30f172611cab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_df.loc[index, "Lat"]=response["results"][0]["geometry"]["location"]["lat"]
<ipython-input-24-30f172611cab>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_df.loc[in

,POSTCODE,ConfirmedCaseCount,Lat,Lon
0,85003,1534,33.4496,-112.079
1,85004,969,33.4528,-112.069
2,85006,3272,33.4636,-112.054
3,85007,2424,33.4463,-112.089
4,85008,9079,33.466,-112.012


In [46]:
#Convert to Float
Final_df['ConfirmedCaseCount'].astype(float).max()

12717.0

In [7]:
VaccineLocation_df["Lat"] = ""
VaccineLocation_df["Lon"] = ""
VaccineLocation_df

,Manufacturer,Location,Address,City,Lat,Lon
0,Pfizer_BioNTech,The Little Clinic-Fry's,"9039 W Olive Ave, Peoria, AZ 85345",Peoria,,
1,Pfizer; J_J,Mountain Park Health Center - Baseline Clinic,"635 E Baseline Rd, Phoenix, AZ 85042",Phoenix,,
2,"Moderna,​​​​Pfizer_BioNTech",CVS/Pharmacy #00193,"4990 S Arizona Ave, Chandler, AZ 85248",Chandler,,
3,"Moderna,​​​​Pfizer_BioNTech",CVS/Pharmacy #03500,"1625 N 44th St, Phoenix, AZ 85008",Phoenix,,
4,"Moderna,​​​​Pfizer_BioNTech",CVS/Pharmacy #03749,"2840 N Dysart Rd, Goodyear, AZ 85395",Goodyear,,
...,...,...,...,...,...,...
364,​​​​Pfizer_BioNTech,Walgreens #4228,"5125 W Northern Ave, Glendale, AZ 85301",Glendale,,
365,​​​​Pfizer_BioNTech,Walgreens #4909,"2921 N 83rd Ave, Phoenix, AZ 85033",Phoenix,,
366,​​​​Pfizer_BioNTech,Walgreens #5504,"4249 W Glendale Ave, Phoenix, AZ 85051",Phoenix,,
367,​​​​Pfizer_BioNTech,Walgreens #6063,"1825 W Bethany Home Rd, Phoenix, AZ 85015",Phoenix,,


In [8]:
# Run Request 
for index, row in VaccineLocation_df.iterrows():
    target_location = row["Address"].replace(" ","+")
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_location}&key={g_key}"
    
#  make request
    response=requests.get(target_url).json()
  
# Extract latitude and longitude
    VaccineLocation_df.loc[index,"Lat"] = response["results"][0]["geometry"]["location"]["lat"]
    VaccineLocation_df.loc[index,"Lon"] = response["results"][0]["geometry"]["location"]["lng"]
        

In [9]:
# Print DataFrame
VaccineLocation_df

,Manufacturer,Location,Address,City,Lat,Lon
0,Pfizer_BioNTech,The Little Clinic-Fry's,"9039 W Olive Ave, Peoria, AZ 85345",Peoria,33.566,-112.253
1,Pfizer; J_J,Mountain Park Health Center - Baseline Clinic,"635 E Baseline Rd, Phoenix, AZ 85042",Phoenix,33.3767,-112.068
2,"Moderna,​​​​Pfizer_BioNTech",CVS/Pharmacy #00193,"4990 S Arizona Ave, Chandler, AZ 85248",Chandler,33.2337,-111.842
3,"Moderna,​​​​Pfizer_BioNTech",CVS/Pharmacy #03500,"1625 N 44th St, Phoenix, AZ 85008",Phoenix,33.4665,-111.987
4,"Moderna,​​​​Pfizer_BioNTech",CVS/Pharmacy #03749,"2840 N Dysart Rd, Goodyear, AZ 85395",Goodyear,33.4782,-112.342
...,...,...,...,...,...,...
364,​​​​Pfizer_BioNTech,Walgreens #4228,"5125 W Northern Ave, Glendale, AZ 85301",Glendale,33.5525,-112.17
365,​​​​Pfizer_BioNTech,Walgreens #4909,"2921 N 83rd Ave, Phoenix, AZ 85033",Phoenix,33.4808,-112.237
366,​​​​Pfizer_BioNTech,Walgreens #5504,"4249 W Glendale Ave, Phoenix, AZ 85051",Phoenix,33.5379,-112.151
367,​​​​Pfizer_BioNTech,Walgreens #6063,"1825 W Bethany Home Rd, Phoenix, AZ 85015",Phoenix,33.5235,-112.099


0      1534.0
1       969.0
2      3272.0
3      2424.0
4      9079.0
        ...  
397     137.0
399       0.0
402       5.0
404       0.0
406       0.0
Name: ConfirmedCaseCount, Length: 377, dtype: float64

In [56]:
# Create a Vaccine Heat Map
fig = gmaps.figure()

Case_Layer = gmaps.WeightedHeatmap(locations=Final_df[['Lat','Lon']], weights=Final_df['ConfirmedCaseCount'],
                                 dissipating=False, max_intensity=40000, 
                                 point_radius = .1)

fig.add_layer(Case_Layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [61]:
# Create a Marker Layer for Maricopa County Vaccination Sites
Vaccination_Layer = gmaps.symbol_layer(
    locations=VaccineLocation_df[['Lat','Lon']], fill_color='rgba(0, 150, 0, 0.4)', 
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2)

fig = gmaps.figure()
fig.add_layer(Vaccination_Layer)
fig.add_layer(Case_Layer)

fig    


Figure(layout=FigureLayout(height='420px'))